In [1]:
from datetime import datetime
import functools as f
from itertools import product

from sklearn.cluster import SpectralClustering
from numpy.typing import NDArray
import numpy as np

from common_generate_predictions import load_data, grid_search

In [2]:
max_number_clusters = 5
wic_data = True
method = "sc"
llm = "wic"
dataset = "dwug_en"
path_to_data = f"../input/wic-scores/{dataset}"
path_to_gold_data = "../test_data_en.csv"
score_paths = ["wic1", "wic2", "wic3", "wic4", "wic5", "wic6", "wic7"]

In [3]:
def get_clusters(
    adj_matrix: NDArray[np.float64 | np.int32], hyperparameters: dict, seed=456
):
    clustering = SpectralClustering(
        n_clusters=hyperparameters["n_clusters"],
        affinity=hyperparameters["affinity"],
        assign_labels=hyperparameters["strategy"],
        random_state=seed,
    ).fit(adj_matrix)

    return clustering.labels_

In [4]:
def generate_hyperparameters_for_sc(max_number_clusters: int):
    combinations = []
    for ncluster in range(2, max_number_clusters + 1):
        for affinity in ["precomputed"]:
            for strategy in ["kmeans"]:
                combinations.append(
                    {
                        "n_clusters": ncluster,
                        "affinity": affinity,
                        "strategy": strategy,
                    }
                )

    return combinations

In [5]:
generate_hyperparameters_for_sc(max_number_clusters),

([{'n_clusters': 2, 'affinity': 'precomputed', 'strategy': 'kmeans'},
  {'n_clusters': 3, 'affinity': 'precomputed', 'strategy': 'kmeans'},
  {'n_clusters': 4, 'affinity': 'precomputed', 'strategy': 'kmeans'},
  {'n_clusters': 5, 'affinity': 'precomputed', 'strategy': 'kmeans'}],)

In [10]:
quantile = range(0, 7)
fill_diagonal = [True, False]
normalize = [True, False]
hc = product(
    quantile,
    fill_diagonal,
    normalize,
    score_paths,
    generate_hyperparameters_for_sc(max_number_clusters),
)

hyperparameter_combinations = [
    {
        "quantile": q,
        "fill_diagonal": fd,
        "normalize": nm,
        "prompt": sp,
        "model_hyperparameters": mhc,
    }
    for q, fd, nm, sp, mhc in hc
]

In [11]:
print(hyperparameter_combinations)
print(len(hyperparameter_combinations))

[{'quantile': 0, 'fill_diagonal': True, 'normalize': True, 'prompt': 'wic1', 'model_hyperparameters': {'n_clusters': 2, 'affinity': 'precomputed', 'strategy': 'kmeans'}}, {'quantile': 0, 'fill_diagonal': True, 'normalize': True, 'prompt': 'wic1', 'model_hyperparameters': {'n_clusters': 3, 'affinity': 'precomputed', 'strategy': 'kmeans'}}, {'quantile': 0, 'fill_diagonal': True, 'normalize': True, 'prompt': 'wic1', 'model_hyperparameters': {'n_clusters': 4, 'affinity': 'precomputed', 'strategy': 'kmeans'}}, {'quantile': 0, 'fill_diagonal': True, 'normalize': True, 'prompt': 'wic1', 'model_hyperparameters': {'n_clusters': 5, 'affinity': 'precomputed', 'strategy': 'kmeans'}}, {'quantile': 0, 'fill_diagonal': True, 'normalize': True, 'prompt': 'wic2', 'model_hyperparameters': {'n_clusters': 2, 'affinity': 'precomputed', 'strategy': 'kmeans'}}, {'quantile': 0, 'fill_diagonal': True, 'normalize': True, 'prompt': 'wic2', 'model_hyperparameters': {'n_clusters': 3, 'affinity': 'precomputed', 'st

In [8]:
metadata = {
    "fill_diagonal": True,
    "normalize": False,
    "method": method,
    "path_to_gold_data": path_to_gold_data,
    "path_to_data": path_to_data,
    "llm": llm,
    "score_paths": score_paths,
    "dataset": dataset,
    "wic_data": wic_data,
    "path_to_save_results": f"../cv-experiments-lscd-all-hyperparameter/{method}/{llm}/{dataset}",
}

In [ ]:
start_time = datetime.now()

grid_search(
    f.partial(load_data, path_to_data),
    get_clusters,
    hyperparameter_combinations,
    metadata=metadata,
)

print(f"Elapsed time: {datetime.now() - start_time}")